https://medium.com/nlplanet/a-full-guide-to-finetuning-t5-for-text2text-and-building-a-demo-with-streamlit-c72009631887

In [ ]:
!pip install datasets rouge_score nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.6 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=fd1ae5b47e23fbcd3eb75f505ce151d1714bce55c0a8069b68e3542f0a8f76b0
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attemp

In [ ]:
!pip install transformers[torch] accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 6.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:
import numpy as np
import pandas as pd
from datasets import Dataset,load_dataset, load_metric
import rouge_score
import nltk
from sklearn.model_selection import train_test_split
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import nltk
nltk.download('punkt')
import string

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
tokenizer = AutoTokenizer.from_pretrained("google-t5/t5-small")
model = AutoModelForSeq2SeqLM.from_pretrained("google-t5/t5-small")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
dataset = load_dataset("json", data_files="/content/drive/MyDrive/Internship/Wikipedia Task/wiki_data.json")

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['content', 'title'],
        num_rows: 4871
    })
})

In [ ]:
datasets_train_test = dataset["train"].train_test_split(test_size=50)
datasets_train_validation = datasets_train_test["train"].train_test_split(test_size=35)

dataset["train"] = datasets_train_validation["train"]
dataset["validation"] = datasets_train_validation["test"]
dataset["test"] = datasets_train_test["test"]

In [ ]:
prefix = "Generate a title: "
max_input_length = 512
max_target_length = 64

def preprocess_data(examples):


  inputs = [prefix + text for text in examples["content"]]
  model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

  # Setup the tokenizer for targets
  with tokenizer.as_target_tokenizer():
    labels = tokenizer(examples["title"], max_length=max_target_length,
                       truncation=True)

  model_inputs["labels"] = labels["input_ids"]
  return model_inputs

In [ ]:

tokenized_datasets = dataset.map(preprocess_data,
                                                 batched=True)

Map:   0%|          | 0/4786 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3946: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/35 [00:00<?, ? examples/s]

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

In [ ]:
batch_size = 8
model_name = "t5-base-medium-title-generation"
model_dir = f"drive/MyDrive/Models/{model_name}"

# args = Seq2SeqTrainingArguments(
#     model_dir,
#     evaluation_strategy="steps",
#     eval_steps=100,
#     logging_strategy="steps",
#     logging_steps=100,
#     save_strategy="steps",
#     save_steps=200,
#     learning_rate=4e-5,
#     per_device_train_batch_size=batch_size,
#     per_device_eval_batch_size=batch_size,
#     weight_decay=0.01,
#     save_total_limit=3,
#     num_train_epochs=1,
#     predict_with_generate=True,
#     fp16=True,
#     load_best_model_at_end=True,
#     metric_for_best_model="rouge1",
#     report_to="tensorboard"
# )

# Remove the deprecated argument from Seq2SeqTrainingArguments
args = Seq2SeqTrainingArguments(
    model_dir,
    evaluation_strategy="steps",
    eval_steps=100,
    logging_strategy="steps",
    logging_steps=100,
    save_strategy="steps",
    save_steps=200,
    learning_rate=4e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=True,
    load_best_model_at_end=True,
    metric_for_best_model="rouge1",
    report_to="tensorboard"
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer)

In [ ]:
metric = load_metric("rouge")

<ipython-input-16-7154407f1f07>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("rouge")


The repository for rouge contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/rouge.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


In [ ]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip()))
                      for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip()))
                      for label in decoded_labels]

    # Compute ROUGE scores
    result = metric.compute(predictions=decoded_preds, references=decoded_labels,
                            use_stemmer=True)

    # Extract ROUGE f1 scores
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}

    # Add mean generated length to metrics
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id)
                      for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [ ]:
# Function that returns an untrained model to be trained
def model_init():
    return AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

trainer = Seq2SeqTrainer(
    model_init=model_init,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

model.safetensors:   0%|          | 0.00/892M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [ ]:
trainer.train()

Step,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
100,3.546500,2.418084,34.764200,13.333300,35.192700,35.218800,4.628600
200,2.645500,2.161440,39.265300,23.027200,39.732400,40.040800,4.342900
300,2.388800,1.978838,43.961700,28.741500,44.494700,44.795900,7.828600
400,2.281000,1.905842,45.631400,28.761900,45.781100,46.136700,7.914300
500,2.214000,1.877392,45.881900,27.585000,45.971100,46.070500,7.542900


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1168: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight', 'lm_head.weight'].


TrainOutput(global_step=599, training_loss=2.5360610274122233, metrics={'train_runtime': 419.5375, 'train_samples_per_second': 11.408, 'train_steps_per_second': 1.428, 'total_flos': 2893273117655040.0, 'train_loss': 2.5360610274122233, 'epoch': 1.0})

In [ ]:

model_name = "t5-base-medium-title-generation/checkpoint-400"
model_dir = f"drive/MyDrive/Models/{model_name}"

tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForSeq2SeqLM.from_pretrained(model_dir)

max_input_length = 512

In [ ]:
text = """
The family Felidae is part of the Feliformia, a suborder that diverged probably about 50.6 to 35 million years ago into several families.[33] The Felidae and the Asiatic linsangs are considered a sister group, which split about 35.2 to 31.9 million years ago.[34]

The earliest cats probably appeared about 35 to 28.5 million years ago. Proailurus is the oldest known cat that occurred after the Eocene–Oligocene extinction event about 33.9 million years ago; fossil remains were excavated in France and Mongolia's Hsanda Gol Formation.[7] Fossil occurrences indicate that the Felidae arrived in North America around 18.5 million years ago. This is about 20 million years later than the Ursidae and the Nimravidae, and about 10 million years later than the Canidae.[35]

In the Early Miocene about 20 to 16.6 million years ago, Pseudaelurus lived in Africa. Its fossil jaws were also excavated in geological formations of Europe's Vallesian, Asia's Middle Miocene and North America's late Hemingfordian to late Barstovian epochs.[36]

In the Early or Middle Miocene, the saber-toothed Machairodontinae evolved in Africa and migrated northwards in the Late Miocene.[37] With their large upper canines, they were adapted to prey on large-bodied megaherbivores.[38][39] Miomachairodus is the oldest known member of this subfamily. Metailurus lived in Africa and Eurasia about 8 to 6 million years ago. Several Paramachaerodus skeletons were found in Spain. Homotherium appeared in Africa, Eurasia and North America around 3.5 million years ago, and Megantereon about 3 million years ago. Smilodon lived in North and South America from about 2.5 million years ago. This subfamily became extinct in the Late Pleistocene.[37]

Results of mitochondrial analysis indicate that the living Felidae species descended from a common ancestor, which originated in Asia in the Late Miocene epoch. They migrated to Africa, Europe and the Americas in the course of at least 10 migration waves during the past ~11 million years. Low sea levels and interglacial and glacial periods facilitated these migrations.[40] Panthera blytheae is the oldest known pantherine cat dated to the late Messinian to early Zanclean ages about 5.95 to 4.1 million years ago. A fossil skull was excavated in 2010 in Zanda County on the Tibetan Plateau.[41] Panthera palaeosinensis from North China probably dates to the Late Miocene or Early Pliocene. The skull of the holotype is similar to that of a lion or leopard.[42] Panthera zdanskyi dates to the Gelasian about 2.55 to 2.16 million years ago. Several fossil skulls and jawbones were excavated in northwestern China.[43] Panthera gombaszoegensis is the earliest known pantherine cat that lived in Europe about 1.95 to 1.77 million years ago.[44]

Living felids fall into eight evolutionary lineages or species clades.[45][46] Genotyping of the nuclear DNA of all 41 felid species revealed that hybridization between species occurred in the course of evolution within the majority of the eight lineages.[47]

Modelling of felid coat pattern transformations revealed that nearly all patterns evolved from small spots.[48]
"""

inputs = ["Generate a short title: " + text]

inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, return_tensors="pt")
output = model.generate(**inputs, num_beams=8, do_sample=True, min_length=10, max_length=20)
decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
predicted_title = nltk.sent_tokenize(decoded_output.strip())[0]

print(predicted_title)


Felidae Species and ancestors


In [ ]:
text = """
Ranim is an intern at saal.ai. She studies IT - multimedia in University of Sharjah.
She is interested in machine learning and deep learning.
She is also interested in data science.
Ranim loves Sara and doesn't want to leave saal.
She always wants to go home early but that's okay.
She is determined and not depresso. Good job Raneem! Hope I get to see you after our internship.
"""

inputs = ["Generate a short title: " + text]

inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, return_tensors="pt")
output = model.generate(**inputs, num_beams=8, do_sample=True, min_length=10, max_length=20)
decoded_output = tokenizer.batch_decode(output, skip_special_tokens=True)[0]
predicted_title = nltk.sent_tokenize(decoded_output.strip())[0]

print(predicted_title)


Ranim Intern at saal.ai


In [ ]:

import torch

# get test split
test_tokenized_dataset = tokenized_datasets["test"]

# pad texts to the same length
def preprocess_test(examples):
  inputs = [prefix + text for text in examples["text"]]
  model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True,
                           padding="max_length")
  return model_inputs

test_tokenized_dataset = test_tokenized_dataset.map(preprocess_test, batched=True)

# prepare dataloader
test_tokenized_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask'])
dataloader = torch.utils.data.DataLoader(test_tokenized_dataset, batch_size=32)

# generate text for each batch
all_predictions = []
for i,batch in enumerate(dataloader):
  predictions = model.generate(**batch)
  all_predictions.append(predictions)

# flatten predictions
all_predictions_flattened = [pred for preds in all_predictions for pred in preds]

# tokenize and pad titles
all_titles = tokenizer(test_tokenized_dataset["title"], max_length=max_target_length,
                       truncation=True, padding="max_length")["input_ids"]

# compute metrics
predictions_labels = [all_predictions_flattened, all_titles]
compute_metrics(predictions_labels)
# {'gen_len': 13.0259,
# 'rouge1': 37.9268,
# 'rouge2': 24.4912,
# 'rougeL': 35.9087,
# 'rougeLsum': 35.9278}
# view rawtitle_prediction_28.py hosted with ❤ by GitHub

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

KeyError: 'text'

In [ ]:

# X_train, X_temp, y_train, y_test = train_test_split(texts, labels, test_size=0.1, random_state=42)


In [ ]:
# dataset = Dataset.from_list(X_train)